# Reconstruction with Truncations
_Author: Lucas Z. Brito_ 

This script performs reconstruction of the HS Hamiltonian including truncations 
of the operator basis.

In [ ]:
using DataFrames
using CSV
using LinearAlgebra
using JSON
using Arpack
using JLD
using Plots

include("./src/hs_tools.jl")


In [ ]:
function proj(v, mat)
	return mat * inv(mat' * mat) * mat' * v
end

function dec_mat(m, n_ops)
	shape = size(m)
	return m[1:(shape[1] - n_ops), 1:(shape[2] - n_ops)]
end

function dec_vec(v, n_ops)
	shape = size(m)
	return v[1:(shape[1] - n_ops)]
end

function decim_recon(mat, coeffs) 
	recons = []; n_ops = Int[]; cs = []; evals = []
	for i in 0:(size(mat)[1] - 1)
		mat_size = size(mat)[1] - i
		eig = eigen(dec_mat(mat, i))
		nullsp = eig.vectors[:, 1:min(2, mat_size)]
		current_evals = eig.values
		c = coeffs[1:end - i]
		evals = vcat(evals, current_evals)
		cs = vcat(cs, c)
		recons = vcat(recons, proj(c, nullsp))
		n_ops = vcat(n_ops, (mat_size) * ones(mat_size))
	end
	return Dict("recons" => recons, "n_ops" => n_ops, "coeffs" => cs, "evals" => evals)
end 


In [ ]:
output_dir = "./data/hs/"

# Obtain WF, CM, FT CM 
n = 21
ft = ft_mat(trunc(Int, n/2))
cm_wf = load("./hs/data/cm_wf_n=$n.jld"); cm = cm_wf["cm"]; k = cm_wf["wf"]
ft_cm = ft * cm * ft'


# Obtain position space and FT HS coefficients
hs_analytical = hs_coeffs(n)
hs_ft = normalize(ft * hs_analytical)

pos_decim_recon = decim_recon(cm, hs_analytical)
recon = pos_decim_recon["recons"]
evals = pos_decim_recon["evals"]

ft_decim_recon = decim_recon(ft_cm, hs_ft)
recon_ft = ft_decim_recon["recons"]
n_ops = ft_decim_recon["n_ops"]
evals_ft_re, evals_ft_im = real(ft_decim_recon["evals"]), imag(ft_decim_recon["evals"])

In [ ]:
# Store reconstruction details in a dataframe.
recon_df = DataFrame(n_ops=n_ops, 
					 recon=recon, 
					 recon_ft_re=real(recon_ft), 
					 recon_ft_im=imag(recon_ft),
					 anal=pos_decim_recon["coeffs"],
					 anal_ft_re=real(ft_decim_recon["coeffs"]),
					 anal_ft_im=imag(ft_decim_recon["coeffs"]),
					 evals=evals, evals_ft_re=evals_ft_re, 
					 evals_ft_im=evals_ft_im)



### Truncations

In [ ]:

# Obtain eigenvalues for each decimation
evals = [eigen(dec_mat(cm, i)).values[1] for i in 0:(size(cm)[1] - 1)]
evals, evals_im = real(evals), imag(evals)
n_ops = [size(cm)[1] - i for i in 0:(size(cm)[1] - 1)]

evals_ft = [eigen(dec_mat(ft_cm, i)).values[1] for i in 0:(size(ft_cm)[1] - 1)]
evals_ft, evals_im_ft = real(evals), imag(evals)

df_evals = DataFrame(n_ops=n_ops, 
			   eval=evals,
			   eval_ft=evals_ft,
			   eval_im=evals_im,
			   eval_im_ft=evals_im_ft)

In [ ]:

df = recon_df # innerjoin(df_evals, recon_df, on=:n_ops)
fn = CSV.write(output_dir * "decim_n=$n.csv", df)
printstyled("Written decimation data to $fn\n", bold=true, color=:green)